In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import pandas as pd
import numpy as np
import cv2
from matplotlib import pyplot as plt
import json

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from random import randint
import time

In [3]:
# decide whether to compute using GPU

device= torch.device("cuda")
#device= torch.device("cpu")
print(device)

In [4]:
fold_0 = pd.read_csv('../input/adience-benchmark-gender-and-age-classification/AdienceBenchmarkGenderAndAgeClassification/fold_0_data.txt',sep='\t')
fold_1 = pd.read_csv('../input/adience-benchmark-gender-and-age-classification/AdienceBenchmarkGenderAndAgeClassification/fold_1_data.txt',sep='\t')
fold_2 = pd.read_csv('../input/adience-benchmark-gender-and-age-classification/AdienceBenchmarkGenderAndAgeClassification/fold_2_data.txt',sep='\t')
fold_3 = pd.read_csv('../input/adience-benchmark-gender-and-age-classification/AdienceBenchmarkGenderAndAgeClassification/fold_3_data.txt',sep='\t')
fold_4 = pd.read_csv('../input/adience-benchmark-gender-and-age-classification/AdienceBenchmarkGenderAndAgeClassification/fold_4_data.txt',sep='\t')
df_combined = pd.concat([fold_0,fold_1,fold_2,fold_3,fold_4],ignore_index=True)
print(f'Number of rows: {len(df_combined)}')
df_combined.info()

In [5]:
df_combined.head()

In [6]:
# new column to specify the file path
#df_combined['filepath'] = 'AdienceBenchmarkGenderAndAgeClassification/faces/' + df_combined['user_id'] + '/coarse_tilt_aligned_face.' + df_combined['face_id'].astype('str') + '.' + df_combined['original_image']
#df_combined['filepath']

df_combined['image_path'] = '../input/adience-benchmark-gender-and-age-classification/AdienceBenchmarkGenderAndAgeClassification/faces/' + df_combined['user_id'] + '/coarse_tilt_aligned_face.' + df_combined['face_id'].astype('str') + '.' + df_combined['original_image']
df_combined['image_path']

In [7]:
from torchvision.io import read_image
from torch.utils.data import Dataset
from skimage import io
from PIL import Image

class FaceDataset(Dataset):
    #def __init__(self, csv_file, model='gender', transform=None):
    def __init__(self, df, model='gender', transform=None):
        #self.df = pd.read_csv(csv_file)
        self.df = df
        self.model = model
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):
        img_path = self.df['image_path'][idx]
        image = Image.open(img_path)
        if self.model == 'gender':
            label = self.df['label_gender'][idx]
        elif self.model == 'age':
            label = self.df['label_age'][idx]
        else:
            print('Please specify "gender" or "age".')
            label = None
        if self.transform:
            image = self.transform(image)
        return (image, label)

In [8]:
transform = transforms.Compose(
    [
        transforms.Resize((128,128)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ]
)

# Age Model

In [9]:
# # age break down
print(df_combined.age.unique())
df_combined.age.value_counts().plot.pie()

In [10]:
# filtered of unlabelled data
print(f'Total Number of records: {len(df_combined)}')
df_age = df_combined.copy()
df_age = df_age[df_age['age'] != 'None']
print(f'Total Number of records with Gender Labels: {len(df_age)}')

# formulating age class
with open('../input/age-class/age_class.json') as f:
    age_class = json.load(f)    
    
df_age['age'] = df_age['age'].apply(lambda x: age_class[x])
df_age.age.value_counts()

# convert class to int
def labelling_age(age_range):
    if age_range == '0-3':
        return 0
    elif age_range == '4-6':
        return 1
    elif age_range == '8-14':
        return 2
    elif age_range == '15-24':
        return 3
    elif age_range == '25-36':
        return 4
    elif age_range == '38-47':
        return 5
    elif age_range == '48-59':
        return 6
    elif age_range == '60+':
        return 7

df_age['label_age'] = df_age['age'].apply(labelling_age)
print('Age Class (after labelling): ')
print(df_age.label_age.value_counts())

df_age.reset_index(inplace=True)
df_age.drop(columns=['index'],inplace=True)
df_age

In [11]:
#df_age.to_csv('/content/drive/MyDrive/MSAI CV/MSAI CV Project/df_age.csv')

In [12]:
#dataset_age = FaceDataset(csv_file='/content/drive/MyDrive/MSAI CV/MSAI CV Project/df_age.csv',model='age',transform=transform)
dataset_age = FaceDataset(df=df_age,model='age',transform=transform)
dataset_age_len = len(dataset_age)
print(dataset_age_len)
print([int(dataset_age_len*0.7),dataset_age_len - int(dataset_age_len*0.7)])
train_age, test_age = torch.utils.data.random_split(dataset_age,[int(dataset_age_len*0.7),dataset_age_len - int(dataset_age_len*0.7)])

from torch.utils.data import DataLoader

train_age_loader = DataLoader(train_age,batch_size=20,shuffle=True)
test_age_loader = DataLoader(test_age,batch_size=20,shuffle=True)

In [13]:
train_features, train_labels = next(iter(train_age_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
plt.imshow(img.permute(1, 2, 0))

In [14]:
class model_age(nn.Module): # VGG_convnet

    def __init__(self):
        super(model_age,self).__init__()

        # block 1:         3 x 128 x 128 --> 64 x 32 x 32        
        self.conv1a = nn.Conv2d(3,   64,  kernel_size=3, padding=1 )
        self.conv1b = nn.Conv2d(64,  64,  kernel_size=3, padding=1 )
        self.pool1  = nn.MaxPool2d(4,4)

        # block 2:         64 x 32 x 32 --> 128 x 8 x 8
        self.conv2a = nn.Conv2d(64,  128, kernel_size=3, padding=1 )
        self.conv2b = nn.Conv2d(128, 128, kernel_size=3, padding=1 )
        self.pool2  = nn.MaxPool2d(4,4)

        # block 3:         128 x 8 x 8 --> 256 x 4 x 4        
        self.conv3a = nn.Conv2d(128, 256, kernel_size=3, padding=1 )
        self.conv3b = nn.Conv2d(256, 256, kernel_size=3, padding=1 )
        self.pool3  = nn.MaxPool2d(2,2)

        #block 4:          256 x 4 x 4 --> 512 x 2 x 2
        self.conv4a = nn.Conv2d(256, 512, kernel_size=3, padding=1 )
        self.pool4  = nn.MaxPool2d(2,2)

        # linear layers:   512 x 2 x 2 --> 2048 --> 4096 --> 4096 --> 10
        self.linear1 = nn.Linear(2048, 4096)
        self.linear2 = nn.Linear(4096, 4096)
        self.linear3 = nn.Linear(4096, 8)
        
    def forward(self, x):

        # block 1:         3 x 32 x 32 --> 64 x 16 x 16
        x = self.conv1a(x)
        x = F.relu(x)
        x = self.conv1b(x)
        x = F.relu(x)
        x = self.pool1(x)

        # block 2:         64 x 16 x 16 --> 128 x 8 x 8
        x = self.conv2a(x)
        x = F.relu(x)
        x = self.conv2b(x)
        x = F.relu(x)
        x = self.pool2(x)

        # block 3:         128 x 8 x 8 --> 256 x 4 x 4
        x = self.conv3a(x)
        x = F.relu(x)
        x = self.conv3b(x)
        x = F.relu(x)
        x = self.pool3(x)

        #block 4:          256 x 4 x 4 --> 512 x 2 x 2
        x = self.conv4a(x)
        x = F.relu(x)
        x = self.pool4(x)

        # linear layers:   512 x 2 x 2 --> 2048 --> 4096 --> 4096 --> 10
        x = x.view(-1, 2048)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x) 
        
        return x

net_age = model_age()
print(net_age)

In [15]:
# send net_age to device
net_age = net_age.to(device)

In [16]:
criterion = nn.CrossEntropyLoss()
my_lr=0.25 
bs= 20

In [17]:
def get_error( scores , labels ):

    bs=scores.size(0)
    predicted_labels = scores.argmax(dim=1)
    indicator = (predicted_labels == labels)
    num_matches=indicator.sum()
    
    return 1-num_matches.float()/bs    

In [18]:
def eval_on_test_set():

    running_error=0
    num_batches=0

    for i in range(0,len(test_age),bs):

        minibatch_data, minibatch_label=  next(iter(test_age_loader))

        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        inputs = minibatch_data

        scores=net_age( inputs ) 

        error = get_error( scores , minibatch_label)

        running_error += error.item()

        num_batches+=1

    total_error = running_error/num_batches
    print( 'error rate on test set =', total_error*100 ,'percent')

In [19]:
start=time.time()

for epoch in range(1,20):
    
    # divide the learning rate by 2 at epoch 10, 14 and 18
    if epoch==10 or epoch == 14 or epoch==18:
        my_lr = my_lr / 2
    
    # create a new optimizer at the beginning of each epoch: give the current learning rate.   
    optimizer=torch.optim.SGD( net_age.parameters() , lr=my_lr )
        
    # set the running quatities to zero at the beginning of the epoch
    running_loss=0
    running_error=0
    num_batches=0
 
    for count in range(0,len(train_age),bs):
    
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch       
        minibatch_data, minibatch_label = next(iter(train_age_loader))
        
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)

        # normalize the minibatch (this is the only difference compared to before!)
        inputs = minibatch_data
        
        # tell Pytorch to start tracking all operations that will be done on "inputs"
        inputs.requires_grad_()

        # forward the minibatch through the net 
        scores=net_age( inputs ) 

        # Compute the average of the losses of the data points in the minibatch
        loss =  criterion( scores , minibatch_label) 
        
        # backward pass to compute dL/dU, dL/dV and dL/dW   
        loss.backward()

        # do one step of stochastic gradient descent: U=U-lr(dL/dU), V=V-lr(dL/dU), ...
        optimizer.step()
        
        # START COMPUTING STATS
        
        # add the loss of this batch to the running loss
        running_loss += loss.detach().item()
        
        # compute the error made on this batch and add it to the running error       
        error = get_error( scores.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches+=1
        # print(f'{num_batches} : {(time.time()-start)/60}')
    
    
    # compute stats for the full training set
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed = (time.time()-start)/60

    print('epoch=',epoch, '\t time=', elapsed,'min','\t lr=', my_lr  ,'\t loss=', total_loss , '\t error=', total_error*100 ,'percent')
    eval_on_test_set() 
    print(' ')